# VGG + LSTM Fall Detection on Videos

In [36]:
%load_ext autoreload
%autoreload 2
%load_ext lab_black

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The lab_black extension is already loaded. To reload it, use:
  %reload_ext lab_black


In [37]:
%cd '/home/jovyan/work/MED_Fall'

/home/jovyan/work/MED_Fall


In [38]:
projectdir = "/home/jovyan/work/MED_Fall"

In [39]:
import absl.logging

absl.logging.set_verbosity(absl.logging.ERROR)

## Imports

In [40]:
# Evaluating CNN+RNN models on the dataset

# Imports
import os
import random
from collections import Counter
from datetime import datetime

import numpy as np
import tensorflow as tf
import wandb
from sklearn.metrics import classification_report
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.models import Sequential
from wandb.keras import WandbCallback

from utils.utility_functions import listdir_nohidden_sorted, safe_mkdir
from vision.vision_utils.cnn_rnn_utils import load_and_split
from vision.vision_utils.dataset import TimeSeriesGenerator

In [41]:
with np.load(
    f"{projectdir}/vision/vision_dataset/vgg_features/Actor_1_Bed_PH.npz"
) as features:
    print(features["arr_0"].shape)

(27720, 512)


In [42]:
os.path.isfile(f"{projectdir}/vision/models/vgg_feature_extractor.h5")

True

In [43]:
model = tf.keras.models.load_model(
    f"{projectdir}/vision/models/vgg_feature_extractor.h5"
)

In [44]:
# os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
# random.seed(hash("setting random seeds") % 2**32 - 1)
# np.random.seed(hash("improves reproducibility") % 2**32 - 1)
# tf.random.set_seed(hash("by removing stochasticity") % 2**32 - 1)

In [45]:
class dotdict(dict):
    """dot.notation access to dictionary attributes"""

    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__


opt = dotdict(
    {
        "lstm1_units": 64,
        "lstm2_units": 32,
        "dense_units": 32,
        "dropout": 0.4,
        "epochs": 1000,
        "train_actors": [4],
        "val_actors": [1],
        "train_cams": [1, 2, 3, 4, 5, 6, 7],
        "val_cams": [1],
        "seq_len": 40,
        "split_ratio": None,
        "drop_offair": True,
        "undersample": False,
        "batch_size": 40,
        "stride": 1,
        "learning_rate": 1e-4,
        "micro_classes": False,
        "num_features": 512,
    }
)

## Load VGG16 pre-trained features

In [46]:
X_train, y_train, X_val, y_val, cams_train, cams_val = load_and_split(
    features_folder=f"{projectdir}/vision/vision_dataset/vgg_features",
    dataset_folder=f"{projectdir}/vision/vision_dataset/ground_truth_new",
    train_actors=opt.train_actors,
    val_actors=opt.val_actors,
    train_cams=opt.train_cams,
    val_cams=opt.val_cams,
    split_ratio=opt.split_ratio,
    drop_offair=opt.drop_offair,
    undersample=opt.undersample,
    micro_classes=opt.micro_classes,
)
print(
    f"\nX_train shape: {X_train.shape}, len y_train: {len(y_train)}, X_val shape: {X_val.shape}, len y_val: {len(y_val)}\n"
)

[STATUS] Load Train Set


  0%|          | 0/6 [00:00<?, ?it/s]

Loading csv datasets:   0%|          | 0/6 [00:00<?, ?it/s]

[STATUS] Load Val Set


  0%|          | 0/16 [00:00<?, ?it/s]

Loading csv datasets:   0%|          | 0/16 [00:00<?, ?it/s]


X_train shape: (39368, 512), len y_train: 39368, X_val shape: (18860, 512), len y_val: 18860



In [47]:
series_gen = TimeSeriesGenerator(opt)
X_train_series, y_train_series, classes = series_gen.get_train_series(
    X_train, y_train, cams_train
)
X_val_series, y_val_series = series_gen.get_val_series(X_val, y_val, cams_val)
print(
    f"\nX_train_series shape: {X_train_series.shape}, len y_train_series: {len(y_train_series)}, X_val_series shape: {X_val_series.shape}, len y_val_series: {len(y_val_series)}\n"
)

100%|██████████| 471/471 [00:00<00:00, 76292.47it/s]


X_train_series shape: (984, 40, 512), len y_train_series: 984, X_val_series shape: (471, 40, 512), len y_val_series: 471



In [48]:
import pandas as pd

pd.Series(y_train_series).value_counts()

0    752
1    203
2     29
dtype: int64

In [49]:
print(f"Before resampling - y_train_series shape: {Counter(y_train_series)}")

Before resampling - y_train_series shape: Counter({0: 752, 1: 203, 2: 29})


## Resample dataset for balancing classes


from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from imblearn.pipeline import Pipeline

undersampling_strategy = "not minority"
# oversampling_strategy = {0: 23890, 2: 6437, 1: 4000}
under = RandomUnderSampler(random_state=2, sampling_strategy=undersampling_strategy)
# over = RandomOverSampler(random_state=2, sampling_strategy=oversampling_strategy)
steps = [("u", under)]
pipeline = Pipeline(steps=steps)
# smt = SMOTETomek(random_state=2)

print(f"Original X_train_series shape: {X_train_series.shape}")

X_train_series = np.reshape(X_train_series, (-1, 20 * opt.num_features))
print(f"X_train_series reshaped: {X_train_series.shape}")

X_train_series, y_train_series = pipeline.fit_resample(X_train_series, y_train_series)

X_train_series = np.reshape(X_train_series, (-1, 20, opt.num_features))

print(f"After resampling X_train_series shape: {X_train_series.shape}")
print(f"After resampling - y_train_series count: {Counter(y_train_series)}")

### Init WANDB project

In [57]:
# WANDB project initialization
run = wandb.init(
    project="Fall detection CNN + RNN",
    config={
        "model": "LSTM",
        "epochs": opt.epochs,
        "seq_len": opt.seq_len,
        "num_features": opt.num_features,
        "batch_size": opt.batch_size,
        "stride": opt.stride,
        "loss_function": "sparse_categorical_crossentropy",
        "architecture": "LSTM",
        "train_actors": opt.train_actors,
        "val_actors": opt.val_actors,
        "train_cams": opt.train_cams,
        "val_cams": opt.val_cams,
        "micro_classes": opt.classes,
        "dropout": opt.dropout,
        "lstm1_units": opt.lstm1_units,
        "lstm2_units": opt.lstm2_units,
        "dense_units": opt.dense_units,
        "learning_rate": opt.learning_rate,
        "split_ratio": opt.split_ratio,
        "drop_offair": opt.drop_offair,
        "undersample": opt.undersample,
    },
)

cfg = wandb.config

wandb: ERROR Control-C detected -- Run data was not synced


Problem at: /tmp/ipykernel_191424/2258791502.py 2 <cell line: 2>


KeyboardInterrupt: 

## LSTM Network

In [51]:
model = Sequential()
model.add(
    LSTM(
        units=cfg.lstm1_units,
        input_shape=(cfg.seq_len, cfg.num_features),
        return_sequences=True,
    )
)
model.add(Dropout(cfg.dropout))
model.add(
    LSTM(
        units=cfg.lstm2_units,
        input_shape=(cfg.seq_len, cfg.num_features),
        return_sequences=False,
    )
)
model.add(Dropout(cfg.dropout))
model.add(Dense(units=cfg.dense_units, activation="relu"))
model.add(Dropout(cfg.dropout))
model.add(Dense(units=np.unique(y_train_series, axis=0).shape[0], activation="softmax"))
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=cfg.loss_function,
    metrics=["accuracy"],
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 40, 64)            147712    
_________________________________________________________________
dropout (Dropout)            (None, 40, 64)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense (Dense)                (None, 32)                1056      
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 9

### Callbacks

In [52]:
dir_path = f"{projectdir}/vision/model_checkpoints/LSTM"

model_checkpoint = ModelCheckpoint(
    filepath=f"{dir_path}/VGG_LSTM_best_epoch.h5",
    monitor="val_loss",
    mode="min",
    save_best_only=True,
    save_weights_only=False,
    initial_value_threshold=0.4,
    verbose=1,
    period=10,
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=30,
    verbose=1,
    mode="min",
    min_delta=1e-5,
    cooldown=1,
    min_lr=1e-6,
)

early_stop = EarlyStopping(
    monitor="val_loss",
    patience=200,
    verbose=1,
    mode="min",
)

callbacks = [WandbCallback(), reduce_lr, early_stop]

### Train the model

In [53]:
# Train Model
history = model.fit(
    X_train_series,
    y_train_series,
    validation_data=(X_val_series, y_val_series),
    epochs=cfg.epochs,
    callbacks=callbacks,
    batch_size=cfg.batch_size,
)

2022-08-19 10:33:20.790071: I tensorflow/core/grappler/devices.cc:66] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 1
2022-08-19 10:33:20.790226: I tensorflow/core/grappler/clusters/single_machine.cc:357] Starting new session
2022-08-19 10:33:20.791219: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 21353 MB memory:  -> device: 0, name: NVIDIA RTX A5000, pci bus id: 0000:65:00.0, compute capability: 8.6
2022-08-19 10:33:20.804123: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:1137] Optimization results for grappler item: graph_to_optimize
  function_optimizer: Graph size after: 452 nodes (370), 621 edges (528), time = 5.372ms.
  function_optimizer: function_optimizer did nothing. time = 0.107ms.

Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.
Incomplete shape.


Epoch 1/1000


2022-08-19 10:33:22.809137: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201


 5/25 [=====>........................] - ETA: 0s - loss: 1.0553 - accuracy: 0.6500  

2022-08-19 10:33:23.469537: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


...

Epoch 22/1000
25/25 [==============================] - 0s 16ms/step - loss: 0.0858 - accuracy: 0.9756 - val_loss: 0.0837 - val_accuracy: 0.9830
Epoch 23/1000
25/25 [==============================] - 0s 16ms/step - loss: 0.0717 - accuracy: 0.9776 - val_loss: 0.0732 - val_accuracy: 0.9830
Epoch 24/1000
25/25 [==============================] - 0s 17ms/step - loss: 0.0757 - accuracy: 0.9715 - val_loss: 0.0622 - val_accuracy: 0.9830
Epoch 25/1000
25/25 [==============================] - 0s 17ms/step - loss: 0.0635 - accuracy: 0.9776 - val_loss: 0.0645 - val_accuracy: 0.9830
Epoch 26/1000
25/25 [==============================] - 0s 17ms/step - loss: 0.0602 - accuracy: 0.9776 - val_loss: 0.0663 - val_accuracy: 0.9830
Epoch 27/1000
25/25 [==============================] - 0s 17ms/step - loss: 0.0777 - accuracy: 0.9766 - val_loss: 0.0410 - val_accuracy: 0.9830
INFO:tensorflow:Assets written to: /home/jovyan/work/MED_Fall/wandb/offline-run-20220819_103317-kdwba5ep/files/model-best/assets


INFO:tensorflow:Assets written to: /home/jovyan/work/MED_Fall/wandb/offline-run-20220819_103317-kdwba5ep/files/model-best/assets
wandb: Adding directory to artifact (/home/jovyan/work/MED_Fall/wandb/offline-run-20220819_103317-kdwba5ep/files/model-best)... Done. 0.0s


Epoch 28/1000
25/25 [==============================] - 0s 16ms/step - loss: 0.0720 - accuracy: 0.9776 - val_loss: 0.0619 - val_accuracy: 0.9830
Epoch 29/1000
25/25 [==============================] - 0s 17ms/step - loss: 0.0577 - accuracy: 0.9766 - val_loss: 0.0808 - val_accuracy: 0.9830
Epoch 30/1000
25/25 [==============================] - 0s 17ms/step - loss: 0.0473 - accuracy: 0.9827 - val_loss: 0.0430 - val_accuracy: 0.9830
Epoch 31/1000
25/25 [==============================] - 0s 17ms/step - loss: 0.0548 - accuracy: 0.9746 - val_loss: 0.0310 - val_accuracy: 0.9830
INFO:tensorflow:Assets written to: /home/jovyan/work/MED_Fall/wandb/offline-run-20220819_103317-kdwba5ep/files/model-best/assets


INFO:tensorflow:Assets written to: /home/jovyan/work/MED_Fall/wandb/offline-run-20220819_103317-kdwba5ep/files/model-best/assets
wandb: Adding directory to artifact (/home/jovyan/work/MED_Fall/wandb/offline-run-20220819_103317-kdwba5ep/files/model-best)... Done. 0.0s


Epoch 32/1000
25/25 [==============================] - 0s 16ms/step - loss: 0.0725 - accuracy: 0.9736 - val_loss: 0.0555 - val_accuracy: 0.9830
Epoch 33/1000
25/25 [==============================] - 0s 17ms/step - loss: 0.0780 - accuracy: 0.9756 - val_loss: 0.0438 - val_accuracy: 0.9830
Epoch 34/1000
25/25 [==============================] - 0s 17ms/step - loss: 0.0732 - accuracy: 0.9756 - val_loss: 0.1009 - val_accuracy: 0.9830
Epoch 35/1000
25/25 [==============================] - 0s 16ms/step - loss: 0.0609 - accuracy: 0.9766 - val_loss: 0.0568 - val_accuracy: 0.9830
Epoch 36/1000
25/25 [==============================] - 0s 17ms/step - loss: 0.0500 - accuracy: 0.9827 - val_loss: 0.0481 - val_accuracy: 0.9830
Epoch 37/1000
25/25 [==============================] - 0s 17ms/step - loss: 0.0528 - accuracy: 0.9776 - val_loss: 0.0423 - val_accuracy: 0.9830
Epoch 38/1000
25/25 [==============================] - 0s 17ms/step - loss: 0.0578 - accuracy: 0.9776 - val_loss: 0.0727 - val_accuracy:

### Evaluate the model

In [54]:
val_logits = model.predict(X_val_series, batch_size=60, verbose=1)

8/8 [==============================] - 0s 5ms/step


In [55]:
# free up memory
# del X_train_series
# del y_train_series
# del X_val_series
# del y_val_series

# gc.collect()

### Log metrics to wandb

In [60]:
y_pred_val_classes = np.argmax(val_logits, axis=1).tolist()

cr = classification_report(y_val_series, y_pred_val_classes)

# wandb.log({"Classification Report": cr})

# wandb.sklearn.plot_roc(y_val_series, val_logits, classes)
# wandb.sklearn.plot_class_proportions(y_train_series, y_val_series, classes)
# wandb.sklearn.plot_precision_recall(y_val_series, val_logits, classes)
# wandb.sklearn.plot_confusion_matrix(y_val_series, y_pred_val_classes, classes)
# wandb.join()

### Classification report

In [61]:
print(cr)

              precision    recall  f1-score   support

           0       0.98      0.99      0.99       463
           1       0.00      0.00      0.00         8

    accuracy                           0.98       471
   macro avg       0.49      0.50      0.49       471
weighted avg       0.97      0.98      0.97       471

